In [14]:
import pandas as pd
import awswrangler as wr
from application.modeling import helper
from application.modeling.constants import MY_SESSION, S3_CLIENT, FileStage, IntermediateFiles

pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:,.2f}".format

In [17]:
bucket_name = "claxon-actuaries-budgeting"
project_id = 11
file_stage = FileStage.intermediate
file_name = IntermediateFiles.salaries_and_pension_and_statutory_contributions_df
df = wr.s3.read_parquet(
    f"s3://{bucket_name}/project_{project_id}/{file_stage.value}/{file_name.value}.parquet",
    boto3_session=MY_SESSION,
)

df

,agent_commission,credit_officer_salaries,credit_officer_commission,other_staff_salaries,total,pensions_and_statutory_contributions
Jan-2024,"7,200.00",600.00,"3,600.00","17,000.00","28,400.00","1,999.36"
Feb-2024,"9,000.00",600.00,"3,840.00","17,000.00","30,440.00","1,999.36"
Mar-2024,"12,240.00",600.00,"4,320.00","17,000.00","34,160.00","1,999.36"
Apr-2024,"14,800.00",600.00,"3,420.00","17,000.00","35,820.00","1,999.36"
May-2024,"17,760.00",900.00,"5,130.00","17,000.00","40,790.00","2,033.44"
Jun-2024,"19,380.00",900.00,"5,400.00","17,000.00","42,680.00","2,033.44"
Jul-2024,"27,300.00","1,200.00","6,000.00","17,000.00","51,500.00","2,067.52"
Aug-2024,"29,250.00","1,400.00","7,700.00","17,000.00","55,350.00","2,090.24"
Sep-2024,"31,500.00","1,400.00","7,700.00","17,000.00","57,600.00","2,090.24"
Oct-2024,"31,500.00","1,800.00","11,250.00","17,000.00","61,550.00","2,135.68"
